# HW9 part 1: Spell checking and edit distances

Written by Lucas Champollion - Based in part on questions by Kushal Chattopadhyay

Modified by Mia Jacobsen

**----------NB-----------** <br>
*if you are working on ucloud make sure you duplicate this file and work in the duplicate instead - otherwise you might not be able to save your work* <br>
**----------NB-----------**

At the end of this lesson, you will be able to:


- Calculate conditional probabilities using tree-drawing methods and Bayes' rule
- Look up frequencies of English words in the Corpus of Contemporary American English
- Use these frequencies to estimate unigram probabilities of English words within an isolated-word spell checker
- Look up frequencies of English letter-based typos
- Look up frequencies of English word-based bigrams on Google
- Combine these frequencies to estimate the most likely correction of a typo within a context-dependent spell checker
- Calculate edit distances for strings and change the weights associated with various types of edit operations


# 1. Probability quiz



Answer the questions below. They are a bit more involved than in previous assignments. Feel free to use a sheet of paper to write out the formulas and probabilities - I highly suggest this actually.

If you are familiar with Google Sheets or Excel, this is a good opportunity to use it.

**Question 1.1** Bob, as he is prone to do, walks into a dark room with a bag in his hands that contains two coins. On this particular day, both coins are biased but in different directions. Coin A comes up heads 30% of the time. Coin B comes up heads 90% of the time. Bob reaches into his bag and picks a coin at random (so, he has a 50% chance of picking either coin). He throws it and it lands on heads. 

What is the probability that the coin he picked is coin A? Give the result either as a fraction, or as a decimal number (you may round for the first two digits).

$P(\text{coin A } | \text{ heads}) = x$


**Hint**: You can solve this question either by drawing balanced trees or by applying Bayes' theorem, or both (this can help you make the connection and prepare you for the subsequent questions). 

Bayes' theorem (A's and B' do not correspond to coins here, this is the generic version): 

$$ P(A|B) = \frac{P(B|A) \times P(A)}{P(B)}$$

If you decide to apply Bayes' theorem, you'll need to know $P(\text{heads})$, the probability that a randomly chosen coin lands on heads. (This is called the _prior_.) Either coin might land on heads. You can calculate the prior by conditioning on whether Bob has picked coin A or coin B, like this:

$P(\text{heads})=[P(\text{heads}|\text{coin A}) \times P(\text{coin A})] + [P(\text{heads}|\text{coin B}) \times P(\text{coin B})]$

This is called marginalization. It's essentially the same as balancing a tree.


In [ ]:
solution_q1_1 = ...

**Question 1.2** You live in a community within which one out of ten people is infected with the coronavirus at a given point in time (as was the case for people living in New York City at the beginning of the pandemic in Spring 2020). To make things simpler, pretend that the virus occurs randomly across the general population.

$P(\text{virus}) = 0.1$ (This is called the _prevalence_ of the disease.)

You decide to get tested using a coronavirus test that has the following properties: 

- Out of any 100 people in your community who are infected with the virus and who take this test, 70 receive a positive test result; that is, the test correctly detects the virus. The remaining 30 are given a false negative test result, that is, they are told they don't have the virus even though they really do. 

$P(\text{positive test}|\text{virus}) = 0.7$ (This is called the _sensitivity_ of the test.)

- Out of any 100 people in your community who are not infected with the virus and who take this test, 99 receive a negative test result; that is, the test correctly detects the absence of the virus. The remaining person is given a false positive test result, that is, this person is told they have the virus even though they really don't. 

$P(\text{negative test}|\text{no virus}) = 0.99$ (This is called the _specificity_ of the test.)

The coronavirus test comes back negative. What is the probability that you don't have the virus, given this result?

$P(\text{no virus}|\text{negative test}) = x$ (This is called the _posterior probability_.)

**Hint 1**: Drawing trees for this question isn't practical because the trees get very large, so you'll want to apply Bayes' theorem:

$$ P(A|B) = \frac{P(B|A) \times P(A)}{P(B)}$$

This will require you to derive various probabilities from those given above.

**Hint 2**: When you have two events that between them cover the entire space of possibilities (e.g. heads/tails, virus/no virus, positive/negative test), their probabilities always add up to 1. E.g. $P(\text{no virus}) + P(\text{virus}) = 1$. So you can get either probability by subtracting the other one from 1. This is also true for conditional probabilities when the two events in question are on the left of the line and there is the same event on the right of the line: e.g. $P(\text{negative test}|\text{virus}) + P(\text{positive test}|\text{virus}) = 1$. It doesn't work on the right of the line though: in our example, $P(\text{negative test}|\text{virus}) + P(\text{negative test}|\text{no virus}) = 0.3 + 0.99 = 1.29$ which is different from 1.

**Hint 3**: To apply Bayes' theorem, you'll also need to know $P(\text{negative test})$, the probability that a randomly chosen person tests negative regardless of whether they have the virus. (This is called the _prior_.) Someone might test negative either because they really don't have the virus, or because they do but the test doesn't work. So we can calculate the prior by conditioning on whether the person has the virus or not, i.e. by doing marginalization, like this:

$P(\text{negative test})=[P(\text{negative test}|\text{virus}) \times P(\text{virus})] + [P(\text{negative test}|\text{no virus}) \times P(\text{no virus})]$

As in the previous question, marginalization is essentially the same as balancing a tree.

**Background on this question:** In real life, the precise sensitivity and specificity of coronavirus tests depends on many factors, but the specificity will be typically higher than the sensitivity. In any case, these numbers vary a lot depending on the kind of test (with PCR tests being more accurate than antigen tests), and on when and how the sample is taken. We ignore all of this here. If you're curious, see this New York Times article ["How to Think Like an Epidemiologist - Don’t worry, a little Bayesian analysis won’t hurt you."](https://www.nytimes.com/2020/08/04/science/coronavirus-bayes-statistics-math.html) and [this Harvard Health blog post](https://www.health.harvard.edu/blog/which-test-is-best-for-covid-19-2020081020734) for more information.



In [ ]:
solution_q1_2 = ...

**Question 1.3** Bob is working on a book of Existentialist poetry and has hired you as his secretary to correct his spelling. Bob is not around, so you need to make decisions without checking with him. (You are working essentially in the same way as an autocorrect system.) Bob's writing looks a bit strange to you, almost as if he was picking each word out of a bag at random. His spelling, unfortunately, is absolutely terrible. One of his favorite words is "exist". 

You come across a typo in Bob's writing: "exitt". Unfortunately the context doesn't make it clear what he means. You surmise that this could be a typo either for "exist" or for "exit". (To make things simpler, we'll assume that these are the only two things he could have intended to type.) But which one?

On the one hand, Bob is an Existentialist and he likes to talk about existence. So he is much more likely to have intended "exist" than "exit", say three times as likely:

$P(\text{Bob intends "exist"}) = 3/4$

$P(\text{Bob intends "exit"}) = 1/4$

On the other hand, "s" and "t" are not really close to each other on Bob's keyboard, and you know that the "t" on his keyboard sometimes sticks because of an old piece of gum. Since he writes "exist" so often, you have a good idea of how often he mistypes it as "exitt" . You count the number of times in the past that he intended to type "exist" but mistyped it as "exitt" and you find that it's about 10% of the time. That is to say, on average, if Bob tries to type the word "exist" on 100 separate occasions, he will mistype it as "exitt" on ten of these occasions. So you estimate that given that he intended to type "exist", the probability that he types "exitt" is 10%: 

$P(\text{Bob types "exitt"|Bob intends "exist"}) = 1/10$

On the third hand :), suppose that Bob intended to type "exit". Given this, how likely is it that he would have typed "exitt" by mistake? You think of the piece of gum underneath Bob's "t" key and you estimate about 20%:

$P(\text{Bob types "exitt"|Bob intends "exit"}) = 1/5$

Of course, you don't actually know whether Bob intended to write "exist" or "exit". That's what you want to know, since you need to correct his spelling. Based on your assumptions so far, given that Bob wrote "exitt", what is the probability that he meant to write "exist"? (This is called the _posterior probability_.)

$P(\text{Bob intends "exist"|Bob types "exitt"}) = x$

**Hint**: You can solve this problem either via balanced trees or via Bayes' rule. If you use trees, note that the two words correspond to the two coins, but that Bob doesn't pick among them with equal probability. So you'll want to balance already the first level of the tree and not just the second. If you use Bayes' theorem, you'll want to calculate $P(\text{Bob types "exitt"})$ (this is called the _prior probability_) via marginalization as in the previous questions. 

In [ ]:
solution_q1_3 = ...

# 2. Spell check: Isolated-word error correction

Based on a question by Kushal Chattopadhyay

An automatic spelling corrector detects the string "poice" as a typo in a document which contains the following sentence:

*The woman’s melodious poice led her to a life of fame as a singer.*

This exercises retraces the steps of the spelling corrector as it fixes this typo.

**Question 2.1** Create a table including four hypothesized correct words (i.e. candidates for what the user might have intended to type).  <br> For this exercise, we will assume that these hypothesized correct words are "voice", "police", "poise", and "price". The answer of this question will consist of four lists of four strings each. Each list is a row, and each list item is a cell. The first list is already filled out for you as an example. The list items should be the following from left to right in each list:

1. First: the required fix, "insertion", "deletion", or "substitution". What modifications would we need to make to the misspelled word to turn it into the correct spelling? 
- If the mistyped string ("poice") and the intended string have the same length, the required fix is a "substitution". E.g. if someone  means to type "acquire" but writes "akquire", that calls for a substitution.
- If the mistyped string ("poice") is shorter than the string that was intended, the required fix is an "insertion". E.g. if someone means to type "acquire" but writes "aquire", that calls for an insertion. 
- If the mistyped string ("poice") is longer than the string that was intended, the required fix is a "deletion". E.g. if someone means to type "acquire" but writes "ackquire", that calls for a deletion. 


2. Second: the hypothesized correct word, i.e. what the user might have meant to type. This is one of the strings "voice", "police", "poise", and "price".


3. Third: the mistyped portion of the input, given what the user might have meant to type. 
- If the required fix is a substitution, give the letter that has actually been typed. E.g. if "acquire" was mistyped as "akquire", this is "k".
- If the required fix is an insertion, give the letter immediately *before* the letter that would need to be inserted to fix the typo. E.g. if someone mistypes "acquire" as "aquire", this is "a". 
- If the required fix is a deletion, give a sequence of two letters: the letter just before the deletion, and the letter that would need to be deleted to fix the typo. E.g. if someone mistypes "acquire" as "ackquire", this is "ck".

4. Fourth: the intended portion of the input.
- If the required fix is a substitution, give the letter that was intended. E.g. if "acquire" was mistyped as "akquire", this is "c".  
- If the required fix is an insertion, give a sequence of two letters consisting of the letter immediately *before* the letter that would need to be inserted to fix the typo, followed by that letter. E.g. if someone mistypes "acquire" as "aquire", this is "ac". 
- If the required fix is a deletion, give the letter immediately *before* the letter that would need to be deleted to fix the typo. E.g. if someone mistypes "acquire" as "ackquire", this is "c".


You can check how your table looks in the test cell. 

First run the following cell so that the table can be created properly. Then fill in the rows. Each row comes with a test that checks that you used the right format. Finally, run the cell below the checks to generate the table in a nice format.

Remember that the example sentence is:

*The woman’s melodious poice led her to a life of fame as a singer.*

In [ ]:
correctionTable_voice = ["substitution", "voice", "p", "v"] # This row has been filled out for you.

In [ ]:
correctionTable_police = ...

In [ ]:
correctionTable_poise = ...

In [ ]:
correctionTable_price = ...

In [ ]:
# Check out how your table looks!
# NB you may need to !pip install pandas and/or numpy 
import pandas as pd
import numpy as np
correctionTable = pd.DataFrame(np.array([correctionTable_voice,
                                         correctionTable_police,
                                         correctionTable_poise,
                                         correctionTable_price]), 
                               columns=["type of required fix",
                                        "hypothesized correct word", 
                                        "mistyped", 
                                        "intended"])
correctionTable

#### Code Explanation
_The following explanation refers to the above code block._

Using a library called `pandas`, the above segment of code takes the `correctionTable_` lists above and organizes them into a neatly labeled table! The column headers from from the `column=[...]` section of the code. The `np` code is related to a library called `numpy`, which helps make a lot of list operations easier and faster with Python.

**Question 2.2 (5 points)** Next, let's guess which of these three hypothesized correct words is most likely the intended one, based on how common each of the possible corrections are in the English vocabulary. To this end, we'll use a corpus (a large and structured collection of texts; the plural of "corpus" is "corpora").

You will use the Corpus of Contemporary American English (COCA) to calculate the frequencies of each of your words. COCA describes itself as "the only large, genre-balanced corpus of American English. COCA is probably the most widely-used corpus of English, and it is related to many other corpora of English that we have created, which offer unparalleled insight into variation in English. The corpus contains more than one billion words of text (25+ million words each year 1990-2019) from eight genres: spoken, fiction, popular magazines, newspapers, academic texts, and (with the update in March 2020): TV and Movies subtitles, blogs, and other web pages." 

To access COCA, go to [https://www.english-corpora.org/coca/](https://www.english-corpora.org/coca/) and select "Corpus of Contemporary American English (COCA)". When you access COCA the first time, you will need to register, which is free. x

1. To find the frequency of a word, type it into the COCA search box and click "Find matching strings", then look under "FREQ". For example, the word "voice" has a frequency of 156,861.

2. To estimate the probability of a word, you need its frequency and the total number of words in the corpus. As of the time of writing, the number of total words in COCA is 1,001,610,938, about one billion. Use this number in this assignment. (For reference, we have looked this up by clicking on the page icon in the top row whose tooltip is ’See texts and registers’ and looking at the rightmost cell in the top row of the table there.).

3. For each word, estimate its unigram probability by dividing its frequency by the total number of words in the corpus. Since the resulting values are very small, scale them by multiplying them by one thousand (1,000), then round the result to three decimals. For example, the word "voice" has an estimated probability of ${156,861\over1,001,610,938}= 0.000156608$. This number multiplied by one thousand, rounded to three decimals, is 0.157. As long as you do this scaling consistently for every word, this will not affect the result of the upcoming computations.

Create a table of probabilities $P(intended)$. The columns of this table should be: 

1. hypothesized correct word
2. frequency of hypothesized correct word
3. P(hypothesized correct word). 

Generate your table in a nice format by running the code below the following cell and its check.

Remember that the example sentence is:

*The woman’s melodious poice led her to a life of fame as a singer.*

In [ ]:
probabilityTable_voice = ["voice", 156861, 0.157]
probabilityTable_police = ...
probabilityTable_poise = ...
probabilityTable_price = ...

In [ ]:
# Check out how your table looks!

probabilityTable = pd.DataFrame(np.array([probabilityTable_voice,
                                          probabilityTable_police,
                                          probabilityTable_poise,
                                          probabilityTable_price]), 
                                columns=["intended word", 
                                         "frequency of this word", 
                                         "scaled unigram probability of this word"])
probabilityTable

<!-- BEGIN QUESTION -->

**Question 2.3** If you were to base your decision solely on these unigram probabilities, which of your hypothesized correct words would come out as most likely to be correct? That is to say, which of them has the highest value in the rightmost column? Did this match what you expect to be the most likely intended word? If so, why do you think this approach works? If not, why do you think the guess failed?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

**Question 2.4** Now, we will refine our guess by adding in how common each incorrect letter error is. Create a table for the noisy channel probabilities of the possible corrections. 

To calculate the error model probabilities, use the table at http://norvig.com/ngrams/count_1edit.txt, which is derived by Peter Norvig from the Google Web Trillion Word Corpus. You can search through the table by using your browser's search function (usually Cmd-F on Mac and Ctrl-F on Windows and Unix). Here is how to read this table:

- An entry like "e|i	917" means that 917 times in the corpus, there was a typo in which the letter "e" was mistyped (seen) where the letter "i" would have been intended (correct). 
- An entry like "t|te	478" stands for a typo in which the letter "e" should have been typed after the letter "t" but was left out by the user, and says that this happened 478 times. 
- An entry like "re|r	299" stands for a typo in which a superfluous letter "e" was inserted after "r" by the user, and says that this happened 299 times. 

This is the same format as you used in the "mistyped" and "intended" columns in Question 2.2 above.

As a quick-and-dirty way to estimate probabilities, take each of these numbers and divide it by 1000. This number is chosen somewhat arbitrarily; as before, it doesn't matter for the computations to follow, as long as it is applied consistently. If a specific transition doesn't occur in Norvig's file, assume that it has happened exactly once (so divide 1 by 1000). This is called "smoothing".

Create a table of error model probabilities $P(\text{mistyped letter}|\text{correct letter})$. The columns of this table should be: 

1. hypothesized correct word

2. a string of the form "letter|intended" (from Peter Norvig's data)

3. Frequency of this letter typo (estimated from Peter Norvig's data as described above)

4. Scaled unigram probability of this word (copied from question 2.2 above)

5. the result of multiplying the last two columns

As before, generate your table in a nice format by running the code below the check. 

Remember that the example sentence is:

*The woman’s melodious poice led her to a life of fame as a singer.*

In [ ]:
noisyChannelTable_voice = ["voice",  "p|v", 0.001, 0.157, 0.001*0.157]
noisyChannelTable_police = ...
noisyChannelTable_poise = ...
noisyChannelTable_price = ...

In [ ]:
# Check out how your table looks!

noisyChannelTable = pd.DataFrame(np.array([noisyChannelTable_voice,
                                           noisyChannelTable_police,
                                           noisyChannelTable_poise,
                                           noisyChannelTable_price]), 
                                 columns=["hypothesized correct word", 
                                          "mistyped letter | intended letter",
                                          "Frequency of this letter typo",
                                          "Scaled unigram probability of this word", 
                                          "Multiplying the last two columns"])
noisyChannelTable

<!-- BEGIN QUESTION -->

**Question 2.5** Which word did you find had the highest noisy channel probability on this approach? That is to say, which one has the highest value in the rightmost column? Did this match your expectations? If so, why do you think this approach works? If not, why do you think the guess failed? What does this tell you about the limits of isolated-word error correction?


_Type your answer here, replacing this text._